## Anaconda package download dashboard

In order to create a download dashboard, we'll first get the last month of download data using ``intake`` as described in the previous notebook.

In [ ]:
import datetime
import intake


cat_url = 'https://raw.githubusercontent.com/ContinuumIO/anaconda-package-data/master/catalog/anaconda_package_data.yaml'
cat = intake.Catalog(cat_url)

today = datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)
try:
    monthly = cat.anaconda_package_data_by_month(year=last_month.year, month=last_month.month).to_dask()
    month = last_month
except:
    # if the last month isn't available, get the month before
    month_before = last_month.replace(day=1) - datetime.timedelta(days=1)
    monthly = cat.anaconda_package_data_by_month(year=month_before.year, month=month_before.month).to_dask()
    month = month_before

package_names = list(monthly.pkg_name.unique())

Next we'll set up a parameterized class to create various plots and widgets to control those plots. We'll use a parameterized class to make it easier to keep track of the computed data.

In [ ]:
import colorcet as cc
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn
import param
pn.extension()


class PackageExplorer(param.Parameterized):
    title = f"## Monthly conda downloads for {month.strftime('%B %Y')}"
    help_text = 'Select a package and optionally a version and groupby column to explore anaconda download data'
    
    package = param.String(default='pandas')
    version = param.Selector(objects=['*.*'], default='*.*')
    groupby = param.Selector(objects=['pkg_python', 'pkg_platform', 'data_source'])
    
    data = param.Parameter(default=None, precedence=-1)
    percent_df = param.Parameter(default=None, precedence=-1)
    conda_badge = param.Parameter(default=None, precedence=-1)
    
    def __init__(self, **params):
        super().__init__(**params)
        self.update_data()
    
    @param.depends('package', watch=True)
    def update_data(self):
        self.data = monthly[monthly['pkg_name'] == self.package].compute()
    
    @param.depends('data', 'version', 'groupby', watch=True)
    def update_percent_df(self):
        data = self.data
        if self.version != '*.*':
            data = data[data['pkg_version'] == self.version]
        tot = data.groupby(self.groupby).counts.sum()
        pct = tot / tot.sum()
        self.percent_df = pd.DataFrame({'percents': pct, 'downloads': tot})
    
    @param.depends('data', watch=True)
    def update_version(self):
        versions = sorted(self.data.pkg_version.unique(), reverse=True)       
        self.param.version.objects = ['*.*'] + versions
        self.version = '*.*'
        
    @param.depends('percent_df', watch=True)
    def update_conda_badge(self, top_of_colormap=1e6):
        colors = cc.palette_n.rainbow[-20:80:-1]
        step = len(colors) / np.log10(top_of_colormap)

        downloads = self.percent_df.downloads.sum()
        
        # get color
        if downloads > top_of_colormap:
            color_index = -1
        elif downloads > 0:
            color_index = int(np.log10(downloads) * step)
        else:
            color_index = 0
        color = colors[color_index][1:]
        
        # format downloads string
        if downloads > 1e6:
            downloads = '{}M'.format(int(downloads/1e6))
        elif downloads > 1e3:
            downloads = '{}k'.format(int(downloads/1e3))
        else:
            downloads = int(downloads)

        self.conda_badge = f"https://img.shields.io/badge/conda-{downloads}/month-{color}.svg"
    
    @param.depends('percent_df')
    def table(self):
        df = self.percent_df.copy()
        df['percents'] = df.percents.apply(lambda x: f'{x:.2%}')
        return df

    @param.depends('percent_df')
    def groupby_plot(self):
        df = self.percent_df
        title = f'{self.package}: {self.version} grouped by {self.groupby}'
        return df.hvplot.barh(y='percents', title=title, responsive=True, min_width=300,
                              ylim=(0, 1), xaxis=False, xlabel='', flip_yaxis=True)

    @param.depends('conda_badge')
    def conda_badge_str(self):
        return pn.pane.Str(self.conda_badge, width_policy='max')
        
    @param.depends('conda_badge')
    def conda_badge_md(self):
        return self.conda_badge
    
    @param.depends('data', 'version')
    def time_plot(self):
        data = self.data
        if self.version != '*.*':
            data = data[data['pkg_version'] == self.version]
        df = data.set_index('time').resample('1H').counts.sum()
        title = title=f'{self.package}: {self.version} hourly downloads'
        return df.hvplot(title=title, responsive=True, min_height=300)

Then we'll instantiate the class and create a layout.

In [ ]:
explorer = PackageExplorer(name='')
package_widget = pn.widgets.AutocompleteInput(options=package_names, value='pandas', name='Package name')

dashboard = pn.Column(
    explorer.title,
    pn.Row(
        pn.Column(
            explorer.help_text,
            pn.panel(explorer.param.package, widgets={'package': package_widget}),
            explorer.param.version,
            pn.panel('Groupby', margin=(10, 0, 0, 10)),
            pn.panel(explorer.param.groupby, widgets={'groupby': pn.widgets.RadioButtonGroup})
        ), 
        pn.Row(
            explorer.table,
            pn.panel(explorer.groupby_plot, margin=(25, 0, 0, 10),
                     align='end', height_policy='max', width_policy='max'),
            align='end'),
        height_policy='max'),
    pn.Row(
        explorer.conda_badge_str,
        explorer.conda_badge_md,
        margin=(10, 0)),
    explorer.time_plot,
    width_policy='max',
)
dashboard.servable('Package Download Dashboard')